# Generate counts dataframes for plotting in R

In [52]:
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, fisher_exact, ranksums, ttest_ind
import seaborn as sns
%matplotlib inline


In [32]:
DATA_DIR = '/Users/felixrichter/Documents/patient_data/nicu_patient_data_2/'
DEID_DATA_DIR = '/Users/felixrichter/Dropbox/PhD/nicu_projects/manuscript_preterm/Data/'
MERGED_LOC = DATA_DIR + '2020_09_29/birth_df_2020_09_21.tsv'
dt_cols = ['BIRTH_DATE', 'DEATH_DATE', 'LOS']
birth_df = pd.read_table(MERGED_LOC, low_memory=False, parse_dates=dt_cols, dtype={'ZIP':str})
birth_df['LOS'] = pd.to_timedelta(birth_df['LOS'])


## Counts for rate

In [3]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
# ~analysis_df.lockdown_group.str.contains('precovid'), 
count_df_ga = analysis_df[['PAT_ID']].groupby([analysis_df.GA_category, analysis_df.BIRTH_YEAR]).count().reset_index().rename(columns={'PAT_ID': 'Births'})
count_df_bw = analysis_df[['PAT_ID']].groupby([analysis_df.BW_category, analysis_df.BIRTH_YEAR]).count().reset_index().rename(columns={'PAT_ID': 'Births'})

In [4]:
count_df_ga

,GA_category,BIRTH_YEAR,Births
0,Extremely premature,2012,10
1,Extremely premature,2013,5
2,Extremely premature,2014,10
3,Extremely premature,2015,15
4,Extremely premature,2016,16
5,Extremely premature,2017,12
6,Extremely premature,2018,14
7,Extremely premature,2019,10
8,Extremely premature,2020,5
9,Late term,2012,10


In [5]:
count_df_ga.to_csv(DEID_DATA_DIR + 'count_df_GA.tsv', sep='\t', index=False)
count_df_bw.to_csv(DEID_DATA_DIR + 'count_df_BW.tsv', sep='\t', index=False)

## Counts for Geography

In [6]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
count_df_geo = analysis_df[['PAT_ID']].groupby([analysis_df.Borough, analysis_df.BIRTH_YEAR]).count().reset_index().rename(columns={'PAT_ID': 'Births'})


In [7]:
# percent by year
c = count_df_geo.groupby(['BIRTH_YEAR', 'Borough'])['Births'].sum().rename("count")
pct_df = c / c.groupby(level=0).sum()
pct_df = pct_df.reset_index()

In [8]:
pct_df.to_csv(DEID_DATA_DIR + 'pct_df_GEO.tsv', sep='\t', index=False)

## Counts for NICU admissions and mortality

In [9]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
count_df_total = analysis_df[['PAT_ID']].groupby([analysis_df.BIRTH_YEAR]).count().reset_index().rename(columns={'PAT_ID': 'Births'})
# count_df_total
count_df_total.to_csv(DEID_DATA_DIR + 'count_df_yearly.tsv', sep='\t', index=False)


8    3603
Name: Births, dtype: int64

In [10]:
b2020 = pd.to_numeric(count_df_total.loc[count_df_total.BIRTH_YEAR == 2020, 'Births'].values[0])
b2019 = pd.to_numeric(count_df_total.loc[count_df_total.BIRTH_YEAR == 2019, 'Births'].values[0])
print(b2020)
print(b2019)
print(b2020 - b2019)
(b2019 - b2020)/b2019

3603
3934
-331


0.08413828164717845

In [11]:
# just triple checking that there aren't any babies admitted to both WBN and NICU
print(birth_df.shape)
print(birth_df.Y_MRN.drop_duplicates().shape)

(61778, 21)
(61778,)


In [12]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
count_df_nicu = analysis_df[['PAT_ID']].groupby([analysis_df.BIRTH_YEAR, analysis_df.DEPARTMENT]).count().reset_index().rename(columns={'PAT_ID': 'Births'})

count_df_nicu
c = count_df_nicu.groupby(['BIRTH_YEAR', 'DEPARTMENT'])['Births'].sum().rename("births_pct")
pct_df = c / c.groupby(level=0).sum()
pct_df = pct_df.reset_index()
pct_df['Births'] = count_df_nicu.Births
pct_df.to_csv(DEID_DATA_DIR + 'nicu_births.tsv', sep='\t', index=False)

pct_df

,BIRTH_YEAR,DEPARTMENT,births_pct,Births
0,2012,NICU,0.065553,204
1,2012,WBN,0.934447,2908
2,2013,NICU,0.050423,167
3,2013,WBN,0.949577,3145
4,2014,NICU,0.064380,229
5,2014,WBN,0.935620,3328
6,2015,NICU,0.067374,254
7,2015,WBN,0.932626,3516
8,2016,NICU,0.053313,206
9,2016,WBN,0.946687,3658


In [13]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
count_df_mortality = analysis_df[['PAT_ID']].groupby([analysis_df.BIRTH_YEAR, ~birth_df.DEATH_DATE.isnull()
]).count().reset_index().rename(columns={'PAT_ID': 'Births', 'DEATH_DATE': 'Deceased'})
count_df_mortality
c = count_df_mortality.groupby(['BIRTH_YEAR', 'Deceased'])['Births'].sum().rename("births_pct")
pct_df = c / c.groupby(level=0).sum()
pct_df = pct_df.reset_index()
pct_df['Births'] = count_df_mortality.Births
pct_df.to_csv(DEID_DATA_DIR + 'mortality.tsv', sep='\t', index=False)
pct_df


,BIRTH_YEAR,Deceased,births_pct,Births
0,2012,False,0.997429,3104
1,2012,True,0.002571,8
2,2013,False,0.997585,3304
3,2013,True,0.002415,8
4,2014,False,0.999438,3555
5,2014,True,0.000562,2
6,2015,False,0.996817,3758
7,2015,True,0.003183,12
8,2016,False,0.998188,3857
9,2016,True,0.001812,7


## Monthly counts and medians for NICU admission and mortality

In [14]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
count_df_nicu = analysis_df[['PAT_ID']].groupby([analysis_df.BIRTH_YEAR, analysis_df.BIRTH_MONTH, analysis_df.DEPARTMENT]).count().reset_index().rename(columns={'PAT_ID': 'Births'})

count_df_nicu
c = count_df_nicu.groupby(['BIRTH_YEAR', 'BIRTH_MONTH', 'DEPARTMENT'])['Births'].sum().rename("births_pct")
pct_df = c / c.groupby(level=[0, 1]).sum()
pct_df = pct_df.reset_index()
pct_df['Births'] = count_df_nicu.Births
pct_df['YEAR_2020'] = '2012-2019'
pct_df.loc[pct_df.BIRTH_YEAR == 2020, 'YEAR_2020'] = '2020'
pct_df_nicu = pct_df.groupby(['YEAR_2020', 'BIRTH_MONTH', 'DEPARTMENT'])['births_pct'].median().reset_index()
# pct_df.to_csv(DEID_DATA_DIR + 'nicu_births_monthly.tsv', sep='\t', index=False)
pct_df_nicu



,YEAR_2020,BIRTH_MONTH,DEPARTMENT,births_pct
0,2012-2019,3,NICU,0.062706
1,2012-2019,3,WBN,0.937294
2,2012-2019,4,NICU,0.053257
3,2012-2019,4,WBN,0.946743
4,2012-2019,5,NICU,0.060270
5,2012-2019,5,WBN,0.939730
6,2012-2019,6,NICU,0.063423
7,2012-2019,6,WBN,0.936577
8,2012-2019,7,NICU,0.065668
9,2012-2019,7,WBN,0.934332


In [15]:
## Mortality
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
count_df_mortality = analysis_df[['PAT_ID']].groupby([analysis_df.BIRTH_YEAR, analysis_df.BIRTH_MONTH, ~birth_df.DEATH_DATE.isnull()]).count().reset_index().rename(columns={'PAT_ID': 'Births', 'DEATH_DATE': 'Deceased'})

c = count_df_mortality.groupby(['BIRTH_YEAR', 'BIRTH_MONTH', 'Deceased'])['Births'].sum().rename("births_pct")
pct_df = c / c.groupby(level=[0, 1]).sum()
pct_df = pct_df.reset_index()
pct_df['Births'] = count_df_mortality.Births
pct_df['YEAR_2020'] = '2012-2019'
pct_df.loc[pct_df.BIRTH_YEAR == 2020, 'YEAR_2020'] = '2020'
pct_df_mortality = pct_df.groupby(['YEAR_2020', 'BIRTH_MONTH', 'Deceased'])['births_pct'].median().reset_index()
pct_df_mortality



,YEAR_2020,BIRTH_MONTH,Deceased,births_pct
0,2012-2019,3,False,0.998382
1,2012-2019,3,True,0.003375
2,2012-2019,4,False,0.997403
3,2012-2019,4,True,0.003252
4,2012-2019,5,False,0.998261
5,2012-2019,5,True,0.003035
6,2012-2019,6,False,0.997429
7,2012-2019,6,True,0.003703
8,2012-2019,7,False,0.998467
9,2012-2019,7,True,0.001873


In [16]:
# combine NICU and mortality into single dataframe
pct_df_mortality_clean = pct_df_mortality.loc[pct_df_mortality.Deceased].drop(columns=['Deceased'])
pct_df_nicu_clean = pct_df_nicu.loc[pct_df_nicu.DEPARTMENT == 'NICU'].drop(columns=['DEPARTMENT'])
pct_df = pd.concat([pct_df_mortality_clean, pct_df_nicu_clean], keys=['Mortality', 'NICU admissions']).reset_index().rename(columns={'level_0': 'Outcome'}).drop(columns=['level_1'])
pct_df.to_csv(DEID_DATA_DIR + 'monthly_mortality_and_nicu_lockdownOnly.tsv', sep='\t', index=False)
pct_df

,Outcome,YEAR_2020,BIRTH_MONTH,births_pct
0,Mortality,2012-2019,3,0.003375
1,Mortality,2012-2019,4,0.003252
2,Mortality,2012-2019,5,0.003035
3,Mortality,2012-2019,6,0.003703
4,Mortality,2012-2019,7,0.001873
5,Mortality,2012-2019,8,0.003752
6,Mortality,2012-2019,9,0.003965
7,Mortality,2020,4,0.001832
8,Mortality,2020,6,0.001815
9,Mortality,2020,9,0.002717


## Medians for Length of Stay

In [45]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
analysis_df =analysis_df.loc[analysis_df.DEPARTMENT == 'NICU']
analysis_df['LOS_days'] = analysis_df.LOS.dt.days
analysis_df['YEAR_2020'] = '2012-2019'
analysis_df.loc[analysis_df.BIRTH_YEAR == 2020, 'YEAR_2020'] = '2020'

#  analysis_df.BIRTH_MONTH; mean and median are lower
print(analysis_df[['LOS_days']].groupby([analysis_df.YEAR_2020]).mean().reset_index())
analysis_df[['BIRTH_YEAR', 'YEAR_2020', 'LOS_days']].to_csv(DEID_DATA_DIR + 'LOS_days_NICU.tsv', sep='\t', index=False)


   YEAR_2020   LOS_days
0  2012-2019  16.871530
1       2020  11.484321


## Monthly counts and medians for prematurity groups

In [19]:
# BW_category GA_category
analysis_grouping = 'GA_category'

analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
analysis_df = analysis_df.loc[analysis_df[analysis_grouping] != 'Not recorded']
count_df_ga = analysis_df[['PAT_ID']].groupby([analysis_df.BIRTH_YEAR, analysis_df.BIRTH_MONTH, analysis_df[analysis_grouping]]).count().reset_index().rename(columns={'PAT_ID': 'Births'})

# get percentages
c = count_df_ga.groupby(['BIRTH_YEAR', 'BIRTH_MONTH', analysis_grouping])['Births'].sum().rename("births_pct")
pct_df = c / c.groupby(level=[0, 1]).sum()
pct_df = pct_df.reset_index()
pct_df['Births'] = count_df_ga.Births
# calculate medians
pct_df['YEAR_2020'] = '2012-2019'
pct_df.loc[pct_df.BIRTH_YEAR == 2020, 'YEAR_2020'] = '2020'
pct_df_ga = pct_df.groupby(['YEAR_2020', 'BIRTH_MONTH', analysis_grouping])['births_pct'].median().reset_index()

# fill not-represented rows with 0
# pd.crosstab(pct_df_ga.YEAR_2020, pct_df_ga.GA_category, pct_df_ga.BIRTH_MONTH)
data_dict = pct_df_ga[['YEAR_2020', 'BIRTH_MONTH', analysis_grouping]].to_dict(orient='list')
rows = itertools.product(*data_dict.values())
tmp_df = pd.DataFrame.from_records(rows, columns=data_dict.keys()).drop_duplicates().reset_index(drop=True)
pct_df_ga = pct_df_ga.merge(tmp_df, how='outer').drop_duplicates().reset_index(drop=True).fillna(0)

# monthly_ga.tsv monthly_bw.tsv
pct_df_ga.to_csv(DEID_DATA_DIR + 'monthly_ga.tsv', sep='\t', index=False)

In [20]:
pct_df_ga

,YEAR_2020,BIRTH_MONTH,GA_category,births_pct
0,2012-2019,3,Extremely premature,0.005272
1,2012-2019,3,Late term,0.003619
2,2012-2019,3,Moderate/late premature,0.068054
3,2012-2019,3,Term,0.922039
4,2012-2019,3,Very premature,0.008230
...,...,...,...,...
65,2020,8,Extremely premature,0.000000
66,2020,8,Late term,0.000000
67,2020,8,Very premature,0.000000
68,2020,9,Extremely premature,0.000000


## Statistical tests (GA comparison, NICU admission, mortality)

In [63]:
analysis_grouping = 'GA_category'
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
analysis_df = analysis_df.loc[analysis_df[analysis_grouping] != 'Not recorded']
# analysis_df['YEAR_2020'] = '2012-2019'
# analysis_df.loc[analysis_df.BIRTH_YEAR == 2020, 'YEAR_2020'] = '2020'

## wilcoxon-rank-sum test
# analysis_df = analysis_df[['YEAR_2020', 'GA_WKS']]
GA_pre2020 = analysis_df.loc[analysis_df.BIRTH_YEAR == 2020]['GA_WKS'].values
GA_2020 = analysis_df.loc[analysis_df.BIRTH_YEAR != 2020]['GA_WKS'].values
print(ranksums(pre2020_GA, GA_2020))
print(np.median(GA_pre2020))
print(np.median(GA_2020))
aprint(ttest_ind(pre2020_GA, GA_2020))
print(GA_pre2020.mean())
print(GA_2020.mean())

RanksumsResult(statistic=-4.085144169924083, pvalue=4.4049457470324234e-05)
39.285714285714285
39.42857142857143
Ttest_indResult(statistic=-1.108488464428895, pvalue=0.26765953532830766)
39.17964095797849
39.21430920451639


In [59]:
# 28101-3603

24498

In [ ]:
analysis_df = birth_df.loc[birth_df.lockdown_group != 'precovid']
analysis_df['YEAR_2020'] = '2012-2019'
analysis_df.loc[analysis_df.BIRTH_YEAR == 2020, 'YEAR_2020'] = '2020'
